In [1]:
import yfinance as yf
import pandas as pd
import os
from datetime import datetime, timedelta
import logging
# from src.config import setup_logging

# Configurar o logging
# setup_logging()
logger = logging.getLogger(__name__)


def load_existing_data(file_path: str) -> pd.DataFrame:
    """
    Carrega os dados processados existentes.

    Args:
        file_path: Caminho para o arquivo CSV.
    Returns:
        DataFrame com os dados existentes
    """
    if os.path.exists(file_path):
        logger.info(f"Carregando dados existentes de {file_path}")
        return pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')

    logger.warning(
        f"Arquivo {file_path} nao encontrado. Criando novo DataFrame.")
    return pd.DataFrame()


def save_data(
    df: pd.DataFrame,
    file_path: str,
    format: str = 'parquet'
) -> None:
    """
    Salva os dados processados em um arquivo.

    Args:
        df: DataFrame com os dados processados.
        file_path: Caminho para o arquivo CSV de saida.
    Returns:
        None
    """
    file_format = file_path.split('.')[-1]
    if format != file_format:
        raise ValueError(
            f"""Formato de saida [{format}] diferente do
             formato do arquivo [{file_format}]""")
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))

    logger.info(f"Salvando dados em {file_path}")
    if format == 'parquet':
        df.to_parquet(file_path)
    elif format == 'csv':
        df.to_csv(file_path, index=False)
    else:
        raise ValueError(f"Formato {format} nao suportado.")


def get_last_date(df: pd.DataFrame) -> datetime:
    """
    Obtem a ultima data presente no DataFrame.

    Args:
        df: DataFrame com os dados existentes.
    Returns:
        Ultima data presente no DataFrame.
    """
    if not df.empty:
        last_date = df.index.max()
        logger.info(f"Ultima data existente: {last_date}")
        return last_date

    logger.info("DataFrame vazio. Não há ultima data.")
    return None


def download_new_data(
    symbol: str,
    start_date: str,
    end_date: str
) -> pd.DataFrame:
    """
    Baixa novos dados de acoes do Yahoo Finance.

    Args:
        symbol: Simbolo da acao.
        start_date: Data de inicio.
        end_date: Data final.
    Returns:
        DataFrame com os novos dados.
    """
    logger.info(f"Baixando dados de {symbol} de {start_date} ate {end_date}")
    # Obtendo os dados da ação
    acao = yf.Ticker(symbol)
    historico_precos = acao.history(period="5y")
    historico_precos['Symbol'] = symbol
    historico_precos = historico_precos[['Symbol', 'Open', 'High', 'Low', 'Close', 'Volume']]
    
    historico_precos = historico_precos.loc[start_date:end_date]

    return historico_precos


def append_new_data(
    existing_df: pd.DataFrame,
    new_df: pd.DataFrame,
    output_path: str
) -> None:
    """
    Appenda novos dados ao DataFrame existente e salva no CSV.

    Args:
        existing_df: DataFrame com os dados existentes.
        new_df: DataFrame com os novos dados.
        output_path: Caminho para o arquivo CSV de saida.
    Returns:
        None
    """
    if existing_df.empty:
        combined_df = new_df
        logger.info(
            "DataFrame existente vazio. Utilizando apenas os novos dados.")
    else:
        combined_df = pd.concat([existing_df, new_df]).drop_duplicates(
            subset=['Date']).sort_values('Date')
        logger.info("Novos dados adicionados ao DataFrame existente.")

    # Reset index e salvar
    combined_df.reset_index(drop=True, inplace=True)
    # combined_df.to_csv(output_path, index=False)
    save_data(combined_df, output_path)
    logger.info(f"Dados atualizados salvos em {output_path}")


def validate_data_continuity(df: pd.DataFrame):
    """
    Valida a continuidade dos dados diarios.

    :param df: DataFrame com os dados existentes.
    """
    df = df.sort_index()
    all_days = pd.date_range(start=df.index.min(
    ), end=df.index.max(), freq='B')  # 'B' para dias uteis
    existing_days = df.index
    missing_days = all_days.difference(existing_days)
    if len(missing_days) == 0:
        logger.info("Validacao de continuidade dos dados bem-sucedida.")
    else:
        logger.error(
            f"Faltam dados para as seguintes datas: {missing_days.tolist()}")
        assert False, f"""Faltam dados para as seguintes datas:
        {missing_days.tolist()}"""


def main():
    SYMBOL = 'DIS'
    PROCESSED_DATA_PATH = 'data/processed/DIS_processed.csv'

    # Carregar dados existentes
    existing_df = load_existing_data(PROCESSED_DATA_PATH)

    if not existing_df.empty:
        last_date = get_last_date(existing_df)
        # Definir o proximo dia util apos a ultima data
        start_date = (last_date + timedelta(days=1)).strftime('%Y-%m-%d')
    else:
        # Se nao houver dados, comecar de uma data inicial
        start_date = '2018-01-01'

    # Data final sera a data atual
    end_date = datetime.today().strftime('%Y-%m-%d')

    logger.info(f"Baixando dados de {start_date} ate {end_date}")

    # Baixar novos dados
    new_df = download_new_data(SYMBOL, start_date, end_date)

    if new_df.empty:
        logger.info("Nenhum novo dado para adicionar.")
        return

    # Ajustar DataFrame
    new_df.set_index('Date', inplace=True)

    # Appendar novos dados
    append_new_data(existing_df, new_df, PROCESSED_DATA_PATH)

    # Recarregar os dados combinados para validacao
    combined_df = load_existing_data(PROCESSED_DATA_PATH)
    combined_df.set_index('Date', inplace=True)

    # Validar a continuidade dos dados
    try:
        validate_data_continuity(combined_df)
    except AssertionError as e:
        logger.error(str(e))


In [13]:
# Data final será a data atual
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = '2018-01-01'

# Baixa os dados históricos da ação
df = (
    yf.download('VIVT3.SA', start=start_date, end=end_date, group_by='column')
    .reset_index()
    .droplevel(level=1, axis=1)
)

# Selecionar apenas as colunas relevantes
df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

[*********************100%***********************]  1 of 1 completed


In [14]:
df.head()

Price,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-02 00:00:00+00:00,41.549999,41.549999,40.910000,41.400002,25.884663,6400
1,2018-01-03 00:00:00+00:00,41.439999,41.439999,40.200001,40.200001,25.134392,29900
2,2018-01-04 00:00:00+00:00,40.310001,40.900002,40.130001,40.639999,25.409485,21000
3,2018-01-05 00:00:00+00:00,40.400002,40.889999,40.400002,40.740002,25.472013,4400
4,2018-01-08 00:00:00+00:00,40.250000,41.200001,40.250000,41.169998,25.740864,6800


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1719 entries, 0 to 1718
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   (Date, )               1719 non-null   datetime64[ns, UTC]
 1   (Open, VIVT3.SA)       1719 non-null   float64            
 2   (High, VIVT3.SA)       1719 non-null   float64            
 3   (Low, VIVT3.SA)        1719 non-null   float64            
 4   (Close, VIVT3.SA)      1719 non-null   float64            
 5   (Adj Close, VIVT3.SA)  1719 non-null   float64            
 6   (Volume, VIVT3.SA)     1719 non-null   int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1)
memory usage: 94.1 KB


In [45]:
pd.read_csv("acoes_vivo.csv")

,Date,Symbol,Open,High,Low,Close,Volume
0,2019-11-18 00:00:00-03:00,VIVT3.SA,33.928420,34.638962,33.928420,34.638962,47100
1,2019-11-19 00:00:00-03:00,VIVT3.SA,34.564944,34.564944,33.661962,33.795189,114900
2,2019-11-21 00:00:00-03:00,VIVT3.SA,33.869195,33.935808,33.358493,33.787781,41200
3,2019-11-22 00:00:00-03:00,VIVT3.SA,33.906213,33.980226,33.558342,33.676765,39000
4,2019-11-25 00:00:00-03:00,VIVT3.SA,33.573150,33.780391,33.388113,33.602757,34200
...,...,...,...,...,...,...,...
1240,2024-11-11 00:00:00-03:00,VIVT3.SA,51.900002,53.080002,51.599998,53.000000,1660600
1241,2024-11-12 00:00:00-03:00,VIVT3.SA,52.529999,53.200001,52.009998,52.250000,2365500
1242,2024-11-13 00:00:00-03:00,VIVT3.SA,52.130001,52.500000,51.740002,52.060001,1708900
1243,2024-11-14 00:00:00-03:00,VIVT3.SA,52.060001,52.389999,51.599998,51.599998,1687200
